imprelement the early format to the db firestore

In [16]:
from google.cloud import firestore
import json

db = '(default)'
db = firestore.Client.from_service_account_json("../keys/nico_pk.json",
                                                 database = db)
'OK'
collection_name = "000"
docs = db.collection(collection_name).stream()

for doc in docs:
    print(doc.id)

5


In [4]:
data = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            9.642123621792251,
            45.07094312259065
          ],
          [
            10.012668713975984,
            44.54462989584778
          ],
          [
            10.79555754917098,
            44.52094589702986
          ],
          [
            10.82585891425154,
            44.03933106725921
          ],
          [
            10.83503330439683,
            43.49718474585413
          ]
        ],
        "type": "LineString"
      }
    }
  ]
}

print(data.get('features')[0].get('geometry').get('coordinates'))


[[9.642123621792251, 45.07094312259065], [10.012668713975984, 44.54462989584778], [10.79555754917098, 44.52094589702986], [10.82585891425154, 44.03933106725921], [10.83503330439683, 43.49718474585413]]


In [20]:
locolls = [0,1,2]
lodocs = [['a', 'b', 'c'],['d', 'e', 'f'],['g', 'h', 'i']]
locoords = [[[0, 0], [1, 1], [2, 2]], [[3, 3], [4, 4], [5, 5]], [[6, 6], [7, 7], [8, 8]]]
for coll in range(len(locolls)):
    # Convert the collection identifier to a string
    collection_id = str(locolls[coll])
    
    # Loop through each document in the current collection
    for doc in range(len(lodocs[coll])):
        # Create a reference to the current document in Firestore
        doc_ref = db.collection(collection_id).document(lodocs[coll][doc])
        
        # Prepare the data to be stored in Firestore
        x = {
            "lat": locoords[coll][doc][0],
            "long": locoords[coll][doc][1]
        }
        
        # Set the data in Firestore for the current document
        doc_ref.set(x)

        

New hypothetical format

In [ ]:
from google.cloud import firestore
import json

db = 'tesinapcesc'
db = firestore.Client.from_service_account_json("../keys/nico_pk.json",
                                                 database = db)

In [ ]:
coll = "gps"
lodocs = ['a', 'b', 'c']

for coll in range(len(locolls)):
    # Convert the collection identifier to a string
    collection_id = str(locolls[coll])
    
    # Loop through each document in the current collection
    for doc in range(len(lodocs[coll])):
        # Create a reference to the current document in Firestore
        doc_ref = db.collection(collection_id).document(lodocs[coll][doc])
        
        # Prepare the data to be stored in Firestore
        x = {
            "lat": locoords[coll][doc][0],
            "long": locoords[coll][doc][1]
        }
        
        # Set the data in Firestore for the current document
        doc_ref.set(x)

In [2]:
import pandas as pd
import numpy as np


gps = pd.read_csv("../document/gps_Updated.csv",index_col=0)
# gps = gps.loc[:, ["individual-local-identifier", 
#                   "individual-taxon-canonical-name", 
#                   "sensor-type", 
#                   "location-long", 
                #   "location-lat"]]
gps = gps.drop(columns='event-id')
gps.rename(columns={
    'timestamp': 'time',
    'location-long': 'long',
    'location-lat': 'lat',
    'individual-local-identifier': 'animal-id'
}, inplace=True)
gps.shape


(89869, 4)

In [3]:
rd = pd.read_csv("../document/reference-data.csv")
rd = rd.loc[:, ['animal-id', 'manipulation-comments']]
rd.shape

(127, 2)

In [4]:
merged_df = pd.merge(gps, rd, on='animal-id', how='inner')

# Supponendo che il DataFrame merged_df sia il risultato del merge tra gps e rd
merged_df['coordinates'] = merged_df.apply(lambda row: [row['lat'], row['long']], axis=1)
merged_df = merged_df.drop(columns=['lat', 'long'])
merged_df.rename(columns={
    'animal-id': 'id',
    'manipulation-comments': 'comments',
}, inplace=True)
merged_df = merged_df.loc[:,['id', 'time', 'coordinates', 'comments']]

# Costruzione della struttura per i grafici ragazze

In [36]:
from google.cloud import firestore
import json
import pandas as pd
import numpy as np

db = 'tesinapcesc'
db = firestore.Client.from_service_account_json("../keys/nico_pk.json",
                                                 database = db)

# Recupera tutti i documenti dalla collezione "references"
docs = db.collection("references").stream()

# Prepara una lista per contenere i dati
data = []

# Estrarre i dati dai documenti
for doc in docs:
    doc_id = doc.id
    doc_dict = doc.to_dict()
    mod = doc_dict.get("mod", "")
    tras = doc_dict.get("tras", "")
    data.append([doc_id, mod, tras])

# Creare il DataFrame di pandas
rf = pd.DataFrame(data, columns=["id", "mod", "tras"])
# Determiniamo l'origine degli uccelli basandosi su: 'comments'
rf['Origine'] = rf['tras'].apply(
    lambda x: 'Helgoland' if 'Heligoland' in x else 'Kazan' if 'Kazan' in x else 'Unknown')

# Filtra i dati per includere solo gli uccelli che hanno realizzato lo spostamento
rf = rf[rf['Origine'] != 'Unknown']
rf.drop(columns='tras', inplace=True)
# print(rf.shape)
# rf.head()

# Visualizza il DataFrame
rf.head()

,id,mod,Origine
1,91733A,OLFAC,Helgoland
11,91744A,control,Helgoland
12,91745A,OLFAC,Helgoland
17,91750A,TRIGEM,Helgoland
22,91756A,OLFAC,Helgoland


In [37]:
# Recupera tutte le collezioni
collections = db.collections()

# Prepara una lista per contenere i dati
data = []

# Filtra e processa solo le collezioni che iniziano con '9'
for collection in collections:
    collection_name = collection.id
    if collection_name in rf['id'].values:
        # Recupera tutti i documenti dalla collezione
        docs = db.collection(collection_name).stream()
        for doc in docs:
            doc_id = doc.id
            doc_dict = doc.to_dict()
            lat = doc_dict.get("lat", "")
            long = doc_dict.get("long", "")
            data.append([collection_name, doc_id, lat, long])

# Creare il DataFrame di pandas
df = pd.DataFrame(data, columns=["id", "event-id", "lat", "long"])
df.shape

(42615, 4)

In [38]:
merged = df.merge(rf, on= 'id', how='inner')
merged.head()

,id,event-id,lat,long,mod,Origine
0,91733A,1082636186,54.18600,7.91883,OLFAC,Helgoland
1,91733A,1082636187,54.18567,7.91917,OLFAC,Helgoland
2,91733A,1082636188,54.18533,7.91500,OLFAC,Helgoland
3,91733A,1082636189,54.18583,7.91917,OLFAC,Helgoland
4,91733A,1082636190,54.33383,8.64117,OLFAC,Helgoland


In [31]:
# Funzione che calcola la distanza tra due punti sulla superficie della Terra usando la formula di Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Raggio della Terra in km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon1 - lon2)
    a = np.sin(dphi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

In [40]:
# Utilizziamo come traiettoria migratoria la mediana delle Longitudini
long_traiettoria_migratoria = merged['long'].median()
print("Il punto medio per le Longitudini è:", long_traiettoria_migratoria)

# Calcola la distanza tra ciascun punto e la long media, tenendo conto del segno
merged['Distance_to_mean'] = merged.apply(lambda row: haversine(
    row['lat'], row['long'],
    row['lat'], long_traiettoria_migratoria
) * (-1 if row['long'] < long_traiettoria_migratoria else 1), axis=1)
merged.head()

Il punto medio per le Longitudini è: 32.46667


,id,event-id,lat,long,mod,Origine,Distance_to_mean
0,91733A,1082636186,54.18600,7.91883,OLFAC,Helgoland,-1589.166955
1,91733A,1082636187,54.18567,7.91917,OLFAC,Helgoland,-1589.157919
2,91733A,1082636188,54.18533,7.91500,OLFAC,Helgoland,-1589.438260
3,91733A,1082636189,54.18583,7.91917,OLFAC,Helgoland,-1589.151737
4,91733A,1082636190,54.33383,8.64117,OLFAC,Helgoland,-1537.311976


In [47]:
# Calcola le distanze medie per lat e tipologia di uccello
grouped = merged.groupby(['Origine', 'lat']).agg(
    Distance_mean=('Distance_to_mean', 'mean'),
    Distance_std=('Distance_to_mean', 'std')
).reset_index()
print(grouped.head())

# Calcola i punti medi per ciascuna lat
mean_points = merged.groupby(['Origine', 'lat']).agg(
    long_mean=('long', 'mean')
).reset_index()
print(mean_points.head())

     Origine      lat  Distance_mean  Distance_std
0  Helgoland -1.78617     -32.879923           NaN
1  Helgoland -1.75700     -71.316903           NaN
2  Helgoland -1.75467     -72.336171           NaN
3  Helgoland -1.74483     -66.871631           NaN
4  Helgoland -1.74417     -66.834977           NaN
     Origine      lat  long_mean
0  Helgoland -1.78617   32.17083
1  Helgoland -1.75700   31.82500
2  Helgoland -1.75467   31.81583
3  Helgoland -1.74483   31.86500
4  Helgoland -1.74417   31.86533


,Origine,lat,Longitudine_mean
0,Helgoland,-1.78617,32.17083
1,Helgoland,-1.75700,31.82500
2,Helgoland,-1.75467,31.81583
3,Helgoland,-1.74483,31.86500
4,Helgoland,-1.74417,31.86533


In [ ]:


# Creazione del grafico
fig, ax = plt.subplots(figsize=(14, 7))
#fig.suptitle('Distanze Medie degli Uccelli Traslocati dal Corridoio Migratorio', fontsize=16)
#vedere se aggiungere colonna per olfatto
'''
color_map = {
    'Uccelli di controllo': 'blue',
    'Uccelli con sezione del nervo trigemino': 'green',
    'Uccelli con sezione del nervo olfattivo': 'red'
}'''

for origine in ['Helgoland', 'Kazan']:
    temp_df = grouped[grouped['Origine'] == origine]
    mean_df = mean_points[mean_points['Origine'] == origine]

    # Traccia le linee per tutti i punti
    for bird_type in temp_df['Tipologia di uccello'].unique():
        bird_df = temp_df[temp_df['Tipologia di uccello'] == bird_type]
        ax.errorbar(bird_df['Distance_mean'], bird_df['lat'],
                    xerr=bird_df['Distance_std'], fmt='-o', label=f'{origine} - {bird_type}' )

    # Traccia i punti medi per ciascuna lat
    ax.scatter(mean_df['Longitudine_mean'], mean_df['lat'], label=f'{origine} - Punti medi', s=100)

ax.axvline(x=0, color='green', linestyle='--')  # Linea verticale mediana
ax.set_xlabel('Distanza dal corridoio migratorio (km)')
ax.set_ylabel('lat')
ax.legend()